배치를 직접 나눠서 원핫인코딩 시, 메모리초과 안나도록!

## 1. 결측치처리
**해당 노트북**
+ 전처리방법2 + x결측치삭제 + vae 활용 + validation set 확인

In [1]:
import pandas as pd
import numpy as np
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

### 1.1. 전처리방법2 데이터 가져오기

In [2]:
train = pd.read_parquet('./data/train_preprocess_2.parquet')
# test = pd.read_parquet('./test.parquet')
test = pd.read_parquet('./data/test_preprocess_2.parquet')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

print(train.shape)
print(train.Delay.value_counts())

(1000000, 19)
Not_Delayed    210001
Delayed         45000
Name: Delay, dtype: int64


### 1.2. 남은 결측치 처리 - 삭제

In [3]:
# print(train.isnull().sum())
# print(train.dropna().shape)
# print(train.dropna().isnull().sum())
train = train.dropna(subset=['Estimated_Departure_Time','Estimated_Arrival_Time','Carrier_Code(IATA)','Airline','Carrier_ID(DOT)'])
print(train.isnull().sum())


# 레이블(Delay)을 제외한 결측값이 존재하는 변수들을 unknown으로 대체합니다.
NaN_col = ['Origin_State','Destination_State','Airline','Estimated_Departure_Time', 'Estimated_Arrival_Time','Carrier_Code(IATA)','Carrier_ID(DOT)']

for col in NaN_col:
    # mode = train[col].mode()[0]
    # train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna('Unknown')
print('Done.')

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time         0
Estimated_Arrival_Time           0
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                          0
Carrier_Code(IATA)               0
Carrier_ID(DOT)                  0
Tail_Number                      0
Delay                       520399
dtype: int64
Done.


In [4]:
# test.head()

In [5]:
# 새로운 column 생성
# train['Estimated_Duration'] = train['Estimated_Arrival_Time'] -  train['Estimated_Departure_Time']
# test['Estimated_Duration'] = test['Estimated_Arrival_Time'] -  test['Estimated_Departure_Time']
test['Tail_Number'].nunique()

6493

### 1.3. label & unlabel split  / label_train & label_validation split

#### 1배치 데이터 흐름
1. vae에는 X_train_labeled와 X_unlabeled를 각각 onehot으로 만들어서 합쳐서 넣어주기
2. classifier에는 X_train_labeled를 onehot으로 만든 것 넣어주기


#### 필요한 것
1. labeled와 unlabeled 나누기
2. labeled에서 train과 validation 분리하기
3. X_train_labeld & X_unlabeled 를 이용한 onehot encoding
4. 전체 데이터에 onehot 적용하면 데이터 크기 너무 커지므로, 배치로 처리하기

#### 1.3.1. 데이터 쪼개기

In [6]:
# 1. labeled & unlabeld split
train_labeled , train_unlabeled = train[train['Delay'].notnull()], train[train['Delay'].isnull()]

X_labeled, y_labeled = train_labeled.drop(['ID','Delay'], axis=1), train_labeled[['Delay']]
change_cate2num = {'Not_Delayed':0, "Delayed":1}
y_labeled['Delay'] = y_labeled['Delay'].apply(lambda x : change_cate2num[x])
X_unlabeled = train_unlabeled.drop(['ID','Delay'], axis=1)

print(X_labeled.shape, X_unlabeled.shape)


# 2. train_labeled & val_labeled split
from sklearn.model_selection import train_test_split
X_train_labeled, X_val_labeled, y_train_labeled, y_val_labeled = train_test_split(X_labeled, y_labeled, test_size=0.2, random_state=42)


# 3. X_unlabled 의 크기를 X_train_labeled와 맞춰주기
# X_unlabeled = X_unlabeled.iloc[:len(X_train_labeled),:]
# print(X_unlabeled.shape, X_train_labeled.shape )

(178176, 17) (520399, 17)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13988\2020010435.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_labeled['Delay'] = y_labeled['Delay'].apply(lambda x : change_cate2num[x])


#### 1.3.2. encoder 만들기

In [96]:

# 3. 데이터 정리 & onehotencoding
from sklearn.preprocessing import OneHotEncoder
cate_cols = ['Month', 'Day_of_Month', 'Cancelled', 'Diverted', 'Origin_Airport',
       'Origin_Airport_ID', 'Origin_State', 'Destination_Airport',
       'Destination_Airport_ID', 'Destination_State', 'Airline',
       'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number']


# Airport 2개 삭제함
cate_cols = ['Month', 'Day_of_Month', 'Cancelled', 'Diverted', 
       'Origin_Airport_ID', 'Origin_State', 
       'Destination_Airport_ID', 'Destination_State', 'Airline',
       'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number']

# cate_cols = ['Month', 'Day_of_Month',
#        'Origin_Airport_ID',
#        'Destination_Airport_ID', 
#              'Airline']

numeric_cols = ['Estimated_Departure_Time','Estimated_Arrival_Time','Distance']


## 3.1. VAE 훈련에 쓸 데이터 : X_train_labeled, X_unlabeled
### 3.1.1. 데이터 정리
X_vae_train = pd.concat([X_train_labeled, X_unlabeled])
X_vae_train_cate = X_vae_train[cate_cols]

encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(X_vae_train_cate)


OneHotEncoder(handle_unknown='ignore')

### 1.4. 데이터 만들기

In [64]:
# 필요한 것 : X_train_labeled, y_train_labeled, X_vae_train, cate_cols, numeric_cols
def make_label_batch(batch_num, num_of_batch):
    # 1. 현재 배치 가져오기 - 추후 X_train_labeled를 섞는 과정도 필요
    n = len(X_train_labeled)
    start_loc = n//num_of_batch*(batch_num-1)
    end_loc = n//num_of_batch*batch_num
    
    X_cur_batch = X_train_labeled.iloc[start_loc:end_loc]
    y_cur_batch = y_train_labeled.iloc[start_loc:end_loc]
    
    # 2. category onehot으로 변환하기
    X_sample_category = X_cur_batch[cate_cols]
    X_sample_category = encoder.transform(X_sample_category)
    X_sample_category = X_sample_category.toarray()  # 추가된 코드: X_sample_category를 2차원 배열로 변환 : 희소행렬로 반환되는 onehot encoding 결과를, 일반적인 Numpy 배열로 변환해줌
    
    # 3. numeric은 0~1 사이로 바꿔주기
    X_sample_numeric = X_cur_batch[numeric_cols]
    max_values = X_vae_train[numeric_cols].max()
    X_sample_numeric = np.array(X_sample_numeric /max_values.values) # 0~1사이로 변환
#     print(X_sample_category.shape, X_sample_numeric.shape)
    
    # 4. category & numeric 합치기
    X_sample = np.hstack([X_sample_category,X_sample_numeric])

    # 5. 텐서로 변환하기 : (batch_size, column dim)
    X_sample = torch.tensor(X_sample, dtype=torch.float32)  
    y_sample = torch.tensor(y_cur_batch.values, dtype=torch.float32)
    
    return X_sample, y_sample

In [79]:
# 필요한 것 : X_unlabeled, X_vae_train, cate_cols, numeric_cols
def make_unlabel_batch(batch_num,num_of_batch):
    # 1. 현재 배치 가져오기 - 추후 X_unlabeled 섞는 과정도 필요
    n = len(X_train_labeled) # 임시방편
    start_loc = n//num_of_batch*(batch_num-1)
    end_loc = n//num_of_batch*batch_num
    
    X_cur_batch = X_unlabeled.iloc[start_loc:end_loc]
    
    # 2. category onehot으로 변환하기
    X_sample_category = X_cur_batch[cate_cols]
    X_sample_category = encoder.transform(X_sample_category)
    X_sample_category = X_sample_category.toarray()  # 추가된 코드: X_sample_category를 2차원 배열로 변환 : 희소행렬로 반환되는 onehot encoding 결과를, 일반적인 Numpy 배열로 변환해줌
    
    # 3. numeric은 0~1 사이로 바꿔주기
    X_sample_numeric = X_cur_batch[numeric_cols]
    max_values = X_vae_train[numeric_cols].max()
    X_sample_numeric = np.array(X_sample_numeric /max_values.values) # 0~1사이로 변환
#     print(X_sample_category.shape, X_sample_numeric.shape)
    
    # 4. category & numeric 합치기
    X_sample = np.hstack([X_sample_category,X_sample_numeric])

    # 5. 텐서로 변환하기 : (batch_size, column dim)
    X_sample = torch.tensor(X_sample, dtype=torch.float32)  
    
    return X_sample

In [97]:
# validation X를 tensor로 만들기
def make_validation_tensor():
    # 1. 현재 배치 가져오기 - 추후 X_unlabeled 섞는 과정도 필요
    n = len(X_train_labeled) # 임시방편
    start_loc = n//num_of_batch*(batch_num-1)
    end_loc = n//num_of_batch*batch_num
    
    X_cur_batch = X_val_labeled.copy()
    
    # 2. category onehot으로 변환하기
    X_sample_category = X_cur_batch[cate_cols]
    X_sample_category = encoder.transform(X_sample_category)
    X_sample_category = X_sample_category.toarray()  # 추가된 코드: X_sample_category를 2차원 배열로 변환 : 희소행렬로 반환되는 onehot encoding 결과를, 일반적인 Numpy 배열로 변환해줌
    
    # 3. numeric은 0~1 사이로 바꿔주기
    X_sample_numeric = X_cur_batch[numeric_cols]
    max_values = X_vae_train[numeric_cols].max()
    X_sample_numeric = np.array(X_sample_numeric /max_values.values) # 0~1사이로 변환
#     print(X_sample_category.shape, X_sample_numeric.shape)
    
    # 4. category & numeric 합치기
    X_sample = np.hstack([X_sample_category,X_sample_numeric])

    # 5. 텐서로 변환하기 : (batch_size, column dim)
    X_sample = torch.tensor(X_sample, dtype=torch.float32)  
    
    return X_sample    

In [99]:
X_val_labeled_tensor = make_validation_tensor()
X_val_labeled_tensor.shape

torch.Size([35636, 7377])

### 모델생성

In [141]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Define the VAE model
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim * 2)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu, log_var = torch.chunk(h, 2, dim=1)
        z = self.reparameterize(mu, log_var)
        x_hat = self.decoder(z)
        return x_hat, mu, log_var




In [142]:
# 분류기 생성
class Classifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
#             nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.network(x)


In [ ]:
# VAE LOSS
import torch
import torch.nn.functional as F

def vae_loss(reconstructed_data, original_data, mu, log_var):
    # Reconstruction Loss
    recon_loss = F.binary_cross_entropy(reconstructed_data, original_data, reduction='mean')

    # KL Divergence Loss
    kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    # Total Loss
    total_loss = recon_loss + kl_divergence
    return total_loss


In [145]:
# Hyperparameters
input_dim = X_label_sample.shape[1]
print('input shape : ', X_label_sample.shape[1])
hidden_dim = 256
latent_dim = 128
learning_rate = 0.001

# Initialize the model, optimizer and loss function
device = 'cuda:2'
vae = VAE(input_dim, hidden_dim, latent_dim)
vae.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
reconstruction_loss = nn.BCELoss(reduction='mean')


# classifier 초기화
classifier = Classifier(input_dim = latent_dim, hidden_dim = latent_dim//2 , output_dim= 2)
classifier.to(device)
classification_loss = nn.CrossEntropyLoss()

input shape :  7377


### 모델훈련
현재 갑자기 training이 안되는 문제 발생

In [148]:
from tqdm import tqdm
for epoch in tqdm(range(30)):
    vae.train()
    classifier.train()
    # epoch 마다 data shuffle
#     tmp = pd.concat([X_train_labeled, y_train_labeled], axis=1)
#     tmp = tmp.sample(frac=1).reset_index(drop=True)
#     X_train_labeled = tmp.drop(columns=['Delay'])
#     y_train_labeled = tmp[['Delay']]
    
    # epoch 마다 loss 세기
    loss_dict = {'total_loss':0, 'labeled_loss':0, 'unlabeled_loss':0, 'class_loss':0}
    num_of_batch = 50
    for batch_num in range(1,num_of_batch+1):
        labeled_data, labeled_label = make_label_batch(batch_num, num_of_batch)
        unlabeled_data = make_unlabel_batch(batch_num, num_of_batch)

        optimizer.zero_grad()

        # Labeled data에 대한 VAE 훈련
        reconstructed_labeled_data, mu, log_var = vae(labeled_data.to(device))
        labeled_loss = reconstruction_loss(reconstructed_labeled_data, labeled_data.to(device)) - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

        # Unlabeled data에 대한 VAE 훈련
        reconstructed_unlabeled_data, mu_unlabeled, log_var_unlabeled = vae(unlabeled_data.to(device))
        unlabeled_loss = reconstruction_loss(reconstructed_unlabeled_data, unlabeled_data.to(device)) - 0.5 * torch.sum(1 + log_var_unlabeled - mu_unlabeled.pow(2) - log_var_unlabeled.exp())

        # Labeled data에 대한 Classifier 훈련
        latent_labeled_data = vae.reparameterize(mu, log_var)
        classifier_output = classifier(latent_labeled_data)
        labeled_label_indices = torch.argmax(labeled_label, dim=1).to(device) # 다중 타겟을 원핫으로 변환
        class_loss = classification_loss(classifier_output, labeled_label_indices)

        # 총 Loss 계산 및 업데이트
        total_loss = labeled_loss + unlabeled_loss + class_loss
        total_loss.backward()
        optimizer.step()

        loss_dict['total_loss'] += total_loss.item()
        loss_dict['labeled_loss'] += labeled_loss.item()
        loss_dict['unlabeled_loss'] += unlabeled_loss.item()
        loss_dict['class_loss'] += class_loss.item()

#         if batch_num % 10 ==0:
#             print(batch_num)

    print(f"epoch:{epoch+1}, total:{loss_dict['total_loss']/num_of_batch : .4f}, labeled_loss:{loss_dict['labeled_loss']/num_of_batch : .4f}, unlabeled_loss:{loss_dict['unlabeled_loss']/num_of_batch : .4f}, class_loss: {loss_dict['class_loss']/num_of_batch : .4f}")


  3%|██▊                                                                                | 1/30 [00:58<28:06, 58.17s/it]

epoch:1, total: 774.8362, labeled_loss: 387.0892, unlabeled_loss: 387.0723, class_loss:  0.6747


  7%|█████▌                                                                             | 2/30 [01:56<27:06, 58.11s/it]

epoch:2, total: 774.8364, labeled_loss: 387.0892, unlabeled_loss: 387.0723, class_loss:  0.6749


  7%|█████▌                                                                             | 2/30 [02:09<30:12, 64.74s/it]


KeyboardInterrupt: 

In [87]:
torch.save(vae.state_dict(), './VAE_new.pth') # 모델 저장
torch.save(classifier.state_dict(), './classifier_new.pth') # 모델 저장

### classifier 성능 비교

In [93]:
# 1. train 이용한 경우 validation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier


# 데이터 수치화
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']
X_train_labeled_le = X_train_labeled.copy()
X_val_labeled_le = X_val_labeled.copy()
test_le = test.copy()
for i in qual_col:
    le = LabelEncoder()
    le=le.fit(train[i]) # 사용할 수 있는 전체 X를 이용해서 LE
    X_train_labeled_le[i]=le.transform(X_train_labeled_le[i])
    X_val_labeled_le[i]=le.transform(X_val_labeled_le[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: # train에 없는 label인 경우
            le.classes_ = np.append(le.classes_, label)
    test_le[i]=le.transform(test_le[i])
print('Done.')


# 모델 정의
models = {
    "Extra Trees Classifier": ExtraTreesClassifier(random_state=42),
    "Random Forest Classifier": RandomForestClassifier(random_state=42),
    "Light Gradient Boosting Machine": LGBMClassifier(random_state=42),
    "Decision Tree Classifier": DecisionTreeClassifier(random_state=42),
    "Gradient Boosting Classifier": GradientBoostingClassifier(random_state=42),
    "Ada Boost Classifier": AdaBoostClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
}

# 각 모델의 성능을 비교
for name, model in models.items():
    model.fit(X_train_labeled_le, y_train_labeled)
    y_pred = model.predict_proba(X_val_labeled_le)
    loss = log_loss(y_val_labeled, y_pred)
    print(f"{name}: Log Loss = {loss:.4f}")

Done.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13988\4099449297.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_labeled_le, y_train_labeled)


Extra Trees Classifier: Log Loss = 0.5265


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13988\4099449297.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_labeled_le, y_train_labeled)


Random Forest Classifier: Log Loss = 0.4804


...

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Ada Boost Classifier: Log Loss = 0.6821


C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Logistic Regression: Log Loss = 0.4554


C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [123]:
# 2. semi-supervised 된 신경망 모델의 validation set 예측성능
from sklearn.metrics import log_loss
with torch.no_grad():
    vae.eval()
    classifier.eval()
    ## cuda로 보내기
    X_val_labeled_tensor.to(device)

    with no_grad()
    ## 잠재벡터화
    reconstructed_labeled_data, mu, log_var = vae(X_val_labeled_tensor)
    X_val_latent_labeled_data = vae.reparameterize(mu, log_var)

    ## 예측
    y_pred = classifier(X_val_latent_labeled_data)
    loss = log_loss(y_val_labeled, torch.softmax(y_pred,dim=1).cpu().detach().numpy()) # softmax로 확률로 바꿔줘야!
    print(f" Log Loss = {loss:.4f}")

Logistic Regression: Log Loss = 0.6390


### 전처리방법8 저장

In [ ]:
save_idx = input('몇 번째 전처리 방법인지 정수-정수를 입력하세요 : ')
train_save_name = 'train_preprocess_' + save_idx
test_save_name = 'test_preprocess_' + save_idx
train.to_parquet(f'./data/{train_save_name}.parquet')
test.to_parquet(f'./data/{test_save_name}.parquet')

몇 번째 전처리 방법인지 정수-정수를 입력하세요 : 4
